In [ ]:
# make sure to run this cell to import the external files we need for today
# and load in the appropriate packages

!git clone https://github.com/ccbskillssem/pythonbootcamp.git

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Introduction

Welcome to our final day of PyCamp! Today, you'll be embarking on your final mini-project, which will unify concepts and themes you've learned over the past several days.

## Single-cell in a nutshell

Recall the last biology class that you took: you probably learned at some point the **central dogma of molecular biology**, which is a fancy name for the principle that DNA gives rise to RNA, and RNA gives rise to protein.

Scientists are able to survey DNA, RNA, and proteins using various methods: **sequencing**, the process of recording the nucleic acid sequence of a DNA/RNA molecule, is quite popular due to its relative scalability and cost efficiency.
* DNA sequencing gives us information on the organism's **genome**, akin to the blueprint of its biology. (We will not be discussing this today, sorry!)
* **RNA sequencing** gives us insight into the creation of an organism's biology: what elements of the blueprint are being realized?

**Single-cell RNA sequencing** (scRNA-seq) is an increasingly popular method for capturing and assigning identity (gene origin) to RNA molecules circulating within a single cell. Briefly, tissue samples are dissociated into single cells, and each RNA molecule within a cell is labeled with a barcode of sorts that indicates its cellular origin. The number and identity of RNA molecules per cell is then determined computationally. This method yields high(er) resolution insights into tissue heterogeneity and temporal cellular processes, such as cell differentiation.

> *Note*: On Monday, we said that our goal isn't to teach you specific bioinformatic analyses, and that's still true. **Today's mini-project is not a full pipeline for analyzing single-cell data**: however, you will be much better equipped to learn how to do that once you're done with the mini-project. (Finish the whole mini-project and you'll realize how much is in your grasp right now!)



## What is a cell atlas?

 A **cell atlas** is an analyzed single-cell dataset comprising multiple cell types over some type of biological scale. Cell atlases grant insight into  cellular heterogeneity across anatomical and physiological scales, and they are especially useful for studying healthy development (ex. stem cell differentiation) and disease progression (ex. cancer).

 The [Tabula Muris Senis](https://www.nature.com/articles/s41586-020-2496-1) is a cell atlas of aging mouse tissues, comprised of samples from multiple healthy mouse specimens of varying ages. For our mini-project, we'll use a subset of the cell atlas data and attempt to replicate some of the findings from the linked paper.

## Overview

We want to work the Tabula Muris Senis data because it has both real biological meaning (spoiler alert: gene expression does change with aging!) *and* is natively represented in a tabular format, much like the arrays and DataFrames you've become accustomed to handling in the last couple of days.

We're going to examine the bladder dataset, as the authors found that bladder tissues change quite significantly with age.

> "*The bladder shows pronounced changes in cell-type composition with age: whereas the mesenchymal compartment of this tissue decreases by a factor of three over the lifetime of the mouse, the urothelial compartment increases by a similar amount...*"

This dataset contains two tables:

1. **A metadata table**: This will be a DataFrame that describes auxilary features about our data: for example, what was the age of each mouse? What tissue did our sample come from, and how many RNA molecules were captured in each sample? This information can be quite useful for guiding what we do with the data.
2. **A count matrix**: This will be a DataFrame that describes *gene expression* in each of our cells. This means that we'll have information on how many "instances" of a gene (read: RNA molecules) were found per cell. We should expect that some genes will be highly expressed (many RNA molecules) in some cells, but near absent in others.

Let's go ahead and get started!

## Downloading the data

The Tabula Muris Senis data is hosted on an external server: we can download a local copy of our desired data using a command line tool called `wget`. Although we don't have access to a terminal in the free version of Colab, we can still execute command line commands using **magic commands**.

1. **For a single line, use the `!` line magic command at the beginning of the line**: This tells the notebook that the following line should be run via the comand line.
2. **For multiple lines, use the `%%bash` magic command at the beginning of the cell.** This tells the notebook that all lines following `%%bash` need to be run via the command line.

These commands work in both Colab notebooks and Jupyter notebooks, and they're very convenient for documenting your full analysis workflow. You can read about other useful notebook magic commands [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [ ]:
# run this cell to download: this should only take a few seconds
%%bash
wget https://cells.ucsc.edu/tabula-muris-senis/facs/bladder/exprMatrix.tsv.gz -O bladder-facs.tsv.gz --no-check-certificate --no-verbose
wget https://cells.ucsc.edu/tabula-muris-senis/facs/bladder/meta.tsv -O bladder-facs.meta.tsv --no-check-certificate --no-verbose

# Review & warm-up

We'll begin with a series of review exercises that we'll do together. Afterwards, you'll have time to work on the mini-project.

## Examining the metadata
Yesterday, you learned how you can perform many familiar Excel-like functions using `pandas` DataFrame functions and methods. We'll have you briefly refresh these skills by examining our metadata.

Our metadata is stored in a simple `.tsv` file called `bladder-facs.meta.tsv`, which we'll load in below:

In [ ]:
# try it out:
# use the appropriate pandas function to load in the metadata table and assign it to a variable called metadata_facs
# then use the .head() method to preview the first few rows of the table


In [ ]:
# try it out:
# inspect the number of rows and columns in this table


Each row of `metadata_facs` contains the metadata of a single cell in the dataset. The `index` column contains a unique string "name" for each cell: let's use this as the actual row Index (column of row labels) for the DataFrame.

In [ ]:
# try it out:
# use the appropriate method to set the index of metadata_facs to the index column
# then use .head() to check that it worked


Now, let's review a couple of very useful methods that you'll need for today's mini-project.

Although we know that each row represents one cell, we still don't know how many samples are represented in this dataset, nor do we know the identity and composition (cell number, cell type) of each sample.

`.unique()` and `.value_counts()` are powerful methods that summarize the unique values of a categorical column and the number of value counts associated with those unique values.

* `.unique()` is a method for **Series**: it does not require any input, and it returns an array of unique values contained in the Series. This method works well when you don't need counts, you only need values.
* `.value_counts()` is a method for **DataFrames**: it takes a variable name and returns a Series with aggregated value counts for each unique value. By default, the Series will be sorted in ascending order of counts.
  * **Note**: By default, this method excludes `NA` values from the values to be counted. You can include `NA` values using the `dropna = False` parameter.

In [ ]:
# try it out:
# access the column names of metadata_facs


In [ ]:
# try it out:
# use .unique() to get unique values in mouse.id


In [ ]:
# try it out:
# use .value_counts() to get the number of cells per mouse.id


`.value_counts()` is especially useful if you want to aggregate counts across multiple variables.

There are two columns in `metadata_facs` that will can tell us about cellular identity: `cell_ontology_class` and `free_annotation`. `cell_ontology_class` tells us about the anatomical placement of the cell, and `free_annotation` tells us about the specific cell type.

We'll revisit this later in the project, but for now, let's use this as an opportunity to test out `.value_counts()` for multiple variables.

In [ ]:
# try it out:
# find the number of cells in each sample for each (anatomical) cell type

# bonus:
# this Series would be more readable if each sample's counts were shown ordered
# by mouse.id, rather than ordered by count values. how might you do this?


## Plotting the metadata

For the past couple of days, you've been using `pyplot` to create simple plots from arrays and DataFrame columns.

Let's start off with a quick review: let's say that we want to create a bar plot with the `sex` categorical on the x axis and cell counts for each value of `sex` on the y axis.

We can use a function called `plt.bar()` to create a simple bar plot. `plt.bar()` requires two inputs:
1. `x`: A list, array, or Series with categories to be plotted on the x axis. These will be the labels for each bar.
2. `height`: A list, array, or Series with numerical values to be plotted on the y axis.

Different plotting functions will require different inputs, depending on the type of plot: for example, `plt.scatter()` wouldn't take `height` as an input, it would require `y` as an input.
> It's almost essential to keep the documentation for your plotting package open in a separate tab. Remember, *don't memorize*, just know where to look for answers.

In [ ]:
# try it out:
# use plt.bar() to create the aforementioned plot
# check the y height of each bar against the categorical counts in the previous cell


We can adjust the size of our plots using `plt.figure()` with the `figsize` parameter. `figsize` accepts a tuple of integers in the format of `(width, height)`.

In [ ]:
# try it out:
# copy your plt.bar() command from the previous cell
# set plt.figure() with a figsize tuple of your choice


So far, we've only made plots to help us inspect our data. If we want to make presentation-worthy plots, we need to add labels.

* `plt.title()`: Takes a string as input and labels the top of the plot with the string. Takes an optional `size` parameter (ex. `size = 16`) to adjust the title size.
* `plt.xlabel()` and `plt.ylabel()`: Same as above, but labels the x and y axes respectively.

In [ ]:
# try it out:
# copy your code from the previous cell
# use the plt functions shown above to add plot labels


It's fairly rare that we only want to plot the counts of one categorical. For example, what if we wanted to create a grouped bar chart showing cell counts split by sample sex and age?

It's time for us to introduce `seaborn`, a package that extends the plotting functionalities of `pyplot`.

In [ ]:
# make sure to run this cell!
import seaborn as sns

`seaborn` provides useful functions that can simultaneously aggregate and plot data directly from DataFrames. It's exceptionally powerful for visualizing mixed categorical and numerical data, which is what we want to do today.
<br><br>

<img src = 'https://seaborn.pydata.org/_images/function_overview_8_0.png' align = 'center'>
<br><br>

`seaborn` functions are built with DataFrames in mind. Most functions tend to take similar key inputs:
* `data`: The DataFrame containing your data of interest. **This is always the first input to a `seaborn` function.**
* `x`: A variable to be plotted on the x-axis.
* `y`: A variable to be plotted on the y-axis.
* `hue`: A variable for coloring groups in the plot.

Let's return to our earlier prompt: we want to create a grouped bar chart showing cell counts split by sample sex and age. Let's start by calculating the counts we expect:

In [ ]:
# try it out:
# use the same .value_counts() method to get cell counts for each combination of sex and age



`sns.countplot()` takes a column of values and calculates counts, plotting them as bars. It uses the `hue` parameter to subdivide bars, and we can use it to create our desired subdivided bar chart.

In [ ]:
# try it out:
# use sns.countplot() with two categorical variables: sex and age
# we want to plot sex on the x axis and subdivide bars by age
# make sure to add labels and a title


It seems like cells from 3 month old male samples are overrepresented in this dataset. One reason might be that some samples yielded more cells than others.

Using `sns.countplot()`, create a plot that:
- Shows total cell count per `mouse.id`.
- Uses the optional `palette` parameter with a palette of your choosing: you can view different options [here](https://www.practicalpythonfordatascience.com/ap_seaborn_palette). (We recommend the classic `viridis` or Berkeley blue-and-gold `cividis`!)

> *Bonus*: Use the optional `order` parameter to specify the order of `mouse.id` values: the specimen with the greatest number of total cells should be first, and the rest of the specimens should follow in descending order.

In [ ]:
# try it out:
# create the plot described above
# note: sns.countplot() takes either x or y as input
# x creates a horizontal plot, y creates a vertical plot


# Mini-project: Tabula Muris Senis bladder data

It's time to start on the mini-project!

This project will require you to revisit topics you've learned in the past few days, emphasizing Wednesday and Thursday's content. We'll also introduce concepts and packages you haven't seen before: consider this a bridge from PyCamp to the rest of your work in Python!

**We highly encourage you to work in groups.** TAs will be available to help you as you go, and we'll pause for breaks and lecturer-led Q&A as we go along.

Due to the nature of this project, **this is the only notebook for this session.** If your Colab notebook disconnects at any point, you can restore your progress by using the `Runtime -> Run before` option in the top (horizontal) menu.

## Examining gene expression
The second table in our dataset is a **count matrix**, with gene names on one axis and cell names on the other. Gene expression is denoted by **counts**: each cell will have a different number of counts for each gene, scaling with the number of RNA molecules identified in the cell.

Below, use the appropriate `pandas` function to load in the count matrix (`bladder-facs.tsv.gz`) and assign it to a variable called `facs`, then inspect it with `.head()`.

> Note: `facs` is a large DataFrame, and it may take a few seconds to load.

In [ ]:
### write your code below ###


The first column of `facs` is a column with the label `gene`, containing gene names. The remaining columns are labeled with cell names, containing counts for each `gene`.

We want this count matrix layout to match our metadata table layout, where each cell is represented by one labeled row: thus, we'll need to **transpose** `facs`. We originally learned about transposition for arrays, and it works largely the same for DataFrames: **you will need to look up the appropriate method for this in the `pandas` documentation.**

Here are the steps you need to take after you have the correct function in hand:
1. Set the `gene` column as the row Index: after transposition, `gene` will be used for column labels.
2. Transpose (switch) the axes of the table using the appropriate DataFrame method.

Make sure that you're updating `facs` with each of these changes. If you've done everything correctly, `facs` should have the same number of rows as `metadata_facs`.

In [ ]:
### write your code below ###


This dataset has already undergone standard quality control and normalization steps for scRNA-seq data: that means that we don't need to do the same data cleaning steps that we did with the other mini-projects. Instead, we'll get right to examining gene expression values across different genes.

### Single distributions

Let's start out with a fairly simple task: visualizing the distribution of a single gene.

Single distributions are commonly visualized using histograms, which we've created before using `plt.hist()`. Below, try using the very similar `sns.histplot()` to create a histogram of gene expresssion values for `Fmo5`.

In [ ]:
### write your code below ###


We can also use boxplots to visualize gene expression distributions: copy the code you used above, but replace `sns.histplot()` with `sns.boxplot()`.

In [ ]:
### write your code below ###


### Multiple distributions
There are several different ways to visualize multiple distributions at the same time. We'll use the plot types in the [Fundamentals of Data Visualization](https://clauswilke.com/dataviz/boxplots-violins.html), which are all available in `seaborn`.

As we mentioned before, `seaborn` is built for DataFrames and can make basic assumptions about how to plot your data. This is particularly useful for plotting multiple distributions.

Given a DataFrame of gene expression values, `seaborn` will automatically plot both columns and color them by their column labels.
* We've provided a list of genes called `subset_genes` below: select these columns from `facs` and assign the resultant DataFrame to a new variable called `subset_facs`.
* Provide `subset_facs` as the sole input to `sns.histplot()`.



In [ ]:
subset_genes = ['Fmo5', 'Id3', 'Lgals3', 'Wfdc2', 'Pcolce']

### write your code below ###


As you can see, histograms can become a bit visually busy with multiple distributions. Boxplots & boxplot variants (violin plots, boxen plots, strip plots, and swarm plots) tend to work much better for higher numbers of distributions visualized in the same plot.

Take a moment to see what these plots look like: simply substitute `sns.histplot()` with the functions below.
* `sns.boxplot()`
* `sns.violinplot()`
* `sns.boxenplot()`

The plots below work best for smaller datasets: we recommend using `subset_facs.sample(50)` to avoid warnings about overloading the plot.
* `sns.stripplot()`: This plot works best for mid-sized datasets: we recommend using `subset_facs.sample(500)` to avoid overloading the plot.
* `sns.swarmplot()`: This plot works best for smaller datasets: we recommend using `subset_facs.sample(25)` to avoid overloading the plot.

In [ ]:
### write your code below ###


### Subdivided distributions

Not all genes are expressed uniformly across all cells, especially cells of different types. What if we wanted to examine expression of these genes in different cell types? In order to do this, we'll need to *merge* our two DataFrames together into a super-DataFrame that contains all the information we want to use.

`facs` and `metadata_facs` can be merged using the row Index (row labels) of each DataFrame, which contains shared values (cell names). However, there's one small problem: the `facs` row Index doesn't actually have a name.

In [ ]:
print('metadata_facs:', metadata_facs.index.name)
print('facs:', facs.index.name)

Recall that we can merge by providing a list of DataFrames/Series to `pd.merge()`, or by calling the `.merge()` method on an existing DataFrame. In either case, we need to specify the **merge key**: a column or index that has the same name in each DataFrame.

Below, perform the following operations:
* Update the `facs` Index name to match the `metadata_facs` index name.
* Merge `subset_facs` with `metadata_facs` using `'index'` as the merge key.
  * Save the resultant DataFrame to a new variable called `merged_subset`.

In [ ]:
### write your code below ###


Using `merged_subset`, create a boxplot of gene expression for `Id3`, then subdivide the boxplot by `age` and `free_annotation`.

In [ ]:
### write your code below ###


## Principal component analysis

**Principal component analysis** is a technique commonly used in scRNA-seq data pre-processing, exploration, and visualization.

> *Principal component analysis (PCA) ... is accomplished by linearly transforming the data onto a new coordinate system (principal components) such that the directions capturing the largest variation in the data can be easily identified. Many studies use the first two principal components in order to plot the data in two dimensions and to visually identify clusters of closely related data points.*  - from [Wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis)

### Feature selection

PCAs are typically used to summarize variation and reduce overall dimensionality in the dataset. scRNA-seq count matrices frequently have many thousands of dimensions (genes), with some genes contributing more to variation than others: these are, as you might predict, **highly variable genes** (HVGs). Filtering the count matrix to only contain highly variable genes is a process called **feature selection**

In [ ]:
# identification of highly variable genes is beyond the scope of PyCamp
# so we've provided them for you here (an array of HVG gene names)
hvg = pd.read_csv('pythonbootcamp/day_5/hvg.csv')['genes'].values
hvg

From `facs`, select the columns in `hvg`: assign the resultant DataFrame to a new variable called `facs_hvg`.

In [ ]:
### write your code below ###


### Dimensionality reduction

We need to bring in another package to perform our PCA: `sklearn` is short for **scikit-learn**, a popular Python package used for machine learning applications.

We only need a single function from `sklearn`:
* `sklearn.decomposition.PCA()`: Creates a PCA generator object akin to the random number generator object created by `np.random.default_rng()`. Takes two inputs: the `n_components` parameter specifies the number of **principal components** (PCs) to be returned, and the `random_state` parameter allows you to set a seed value for the pseudo-random aspect of PCA.

Run the cell below to create the PCA generator object: we've set the random state to ensure that your results are comparable to the answer key.

In [ ]:
# make sure to run this cell! nothing will work without it

import sklearn.decomposition
pca = sklearn.decomposition.PCA(n_components = 50, random_state = 7)

`pca` is a generator that will transform a dataset and summarize it into an array of the first 50 principal components. The first principal component (PC1) will summarize the largest amount of variation in the data, the second (PC2) will summarize the second largest amount of variation, and so on and so forth.

We need to use our new `pca` object to fit and transform `facs_hvg`: we can accomplish this by chaining together two methods:
* `.fit()`: Takes a DataFrame or array as input, fits `pca` to the input data.
* `.transform()`: Takes a DataFrame or array as input, transforms the input data according to `pca` into an array. **This method can only be used with the same input provided to `.fit()`**.

In [ ]:
### write your code below ###


This array contains the transformed principal component coordinates of each row in `facs`. However, this array is currently unlabeled, which will make it difficult for us to use in plotting: we need to transform it into a DataFrame with an appropriate index and column names.

Construct a DataFrame from `pca_array` using the `pd.DataFrame()` function, then assign it to a variable called `pca_df`. You will need to provide column labels and an index using the appropriate parameters, which you can find in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

*Hint*: The PCA transformation reduces the number of columns, but the number and order of rows, as well as the row labels, remains unchanged.

In [ ]:
### write your code below ###


## PCA visualization
As Wikipedia mentioned, it's quite common to use the first two principal components for exploratory visualization, as they capture the maximum amount of variation in the dataset in two dimensions.

Start off by creating a basic scatterplot of the first two PCs with `sns.scatterplot()`.
> *Note*: PCA plots typically hide coordinate values and ticks: you can remove x and y ticks on PCA plots by adding `plt.xticks([], [])` and `plt.yticks([], [])`. All plots in the answer key will obey this convention.

In [ ]:
### write your code below ###


### Metadata
PCA plots are typically colored by some metadata variable that may be relevant for understanding why data points cluster together: what if we simply added principal components as extra columns on our metadata table?

We can take advantage of the fact that `metadata_facs`, `facs`, and `facs_df` all use the same index: using the `.merge()` method, we can create a single convenient DataFrame containing the first two PCs and the metadata for each cell.

Select the first two PCs in `pca_df` and join them with `metadata_facs` using `.merge()`, assigning the resultant DataFrame to a variable called `pca_with_metadata`.

In [ ]:
### write your code below ###


Once you're ready, create the PCA scatterplot just as you did before: this time, use the `hue` parameter to color the PCA plot by `cell_ontology_class`.

In [ ]:
### write your code below ###


### Gene expression

Let's take this one step further by coloring our plot with gene expression values from `facs`.

The authors of the Tabula Muris Senis [paper](https://www.nature.com/articles/s41586-020-2496-1) identified the top 20 genes upregulated and downregulated with age in three different tissues, including the bladder (Fig. 2F). Below, we've transcribed the list of genes from the figure for your convenience.

In [ ]:
age_up = ['Sprr1a', 'Ly6d', 'Lgals3', 'Wfdc2', 'Gsta4', 'Gsto1', 'Krt18', 'Sfn',
          'Krt8', 'Areg', 'Aqp3', 'Igfbp2', 'Krt15', 'Gstm1', 'Krt19', 'Fxyd3',
          'Akr1b8', 'Foxq1', 'Cldn4', 'Krt7']

age_down = ['Pcolce', 'Ppic', 'Id3', 'Col6a2', 'Gadd45g', 'Tuba1a', 'Fos', 'Dcn',
            'Col5a2', 'Htra1', 'Car3', 'Rcn3', 'Mgp', 'Lgals1', 'Col6a1',
            'Col1a1', 'Serpinh1', 'Sparc', 'Col1a2', 'Col3a1']

Although our dataset is very high-quality, it's still useful to perform little sanity checks to make sure that everything is as it should be: in this case, we should make sure that all of the genes listed in the figure are actually in `facs`.

Let's take this opportunity to review two core concepts: writing custom functions and using `for` loops.

In [ ]:
# write a custom function called gene_check() that fulfills the following requirements:
# 1) takes two inputs: a list of genes (gene_list), and a DataFrame (df)
# 2) uses a for loop to check if each gene in age_up exists in df
# 3) if the gene is not in df, prints a warning message of your choice that
# includes the gene name

### write your code below ###


In [ ]:
# use this cell to test your function
# none of the genes in control_genes should be found in facs
# all of the genes in age_up and age_down should be found in facs

control_genes = pd.Series(['BRCA1', 'APOE', 'LCT'])
control_genes.index = control_genes

gene_check(control_genes, facs)

Now that we've verified that all of our aging-upregulated and downregulated genes are present in `facs`, we can color our PCA plot by individual genes and their expression values.

Just as you did before, select the first two PCs from `pca_df` and join them with `facs` using `.merge()`, assigning the resultant DataFrame to `pca_with_expression`.

In [ ]:
### write your code below ###


Once you're done, you can color your PCA plot with expression values from `age_up` and `age_down`, just like you did with the metadata.

This excerpt from the article suggests some specific genes of interest:

> "*... using differential gene expression analysis to assess overall changes in tissues with age, we found that stromal-associated genes (Col1a1, Col1a2, Col3a1 and Dcn) are downregulated while epithelial-associated genes (Krt15, Krt18 and Sfn) are upregulated, supporting the compositional observations.*"

Notes:
* Set the `palette` parameter with a continuous palette of your choice: you can view different options [here](https://www.practicalpythonfordatascience.com/ap_seaborn_palette).
* For most palettes, it's advisable to also use the optional `legend = None` and `linewidth = 0` parameters to reduce visual noise.
* Expression values will be plotted with a wide color spectrum: we recommend sorting the DataFrame by the gene's expression values.
  * This ensures that the "brightest" points are plotted last, making it easier to see patterns in the colored plot.

If you want to change additional aspects of your plot, you can view the documentation for `sns.scatterplot()` [here](https://seaborn.pydata.org/generated/seaborn.scatterplot.html).


In [ ]:
### write your code below ###


## Final steps


### Saving plots

`pyplot` offers a convenient function for saving plots. `plt.savefig()` requires a filename (`fname`) and takes an optional `dpi` parameter. We recommend using `dpi = 300` for publication-ready plots.

To save your plot, simply add `plt.savefig()` to the end of your plotting cell, then run the cell.

In [ ]:
### place the code for your favorite plot below ###

### place the code for your favorite plot above ###

plt.savefig('my_favorite_plot.png', dpi = 300)

You can now download this plot from the Files menu. (If it hasn't appeared, try refreshing the Files menu.)

### The ultimate test: comparing to `scanpy`

There are many, many packages available for single-cell data analysis in Python: we just installed `scanpy`, a very popular package that was used in the full Tabula Muris Senis dataset analysis.

Packages like `scanpy` are well-loved because they bundle together many single-cell operations and analyses into individual functions. Below, we've written the `scanpy` commands that replicate each step of what we did above, down to the final PCA plot.

In [ ]:
# we need to use pip to install scanpy first
# this should take < 30 seconds
!pip install scanpy

In [ ]:
import scanpy as sc
import anndata

# creates an object called adata that "holds" both the count matrix and metadata
# it's basically a container for various DataFrames in the dataset
adata = anndata.AnnData(X = facs)
adata.obs = metadata_facs

# calculates highly variable genes and updates adata
sc.pp.highly_variable_genes(adata)

# performs PCA and updates adata
sc.tl.pca(adata)

# creates a PCA plot and colors it by one of our genes of interest
# it automatically sorts gene expression and places brightest points last
sc.pl.pca(adata, color='Krt15')

Hopefully this looks similar to your PCA plot when colored by the same gene. If so, **congratulations**: you've matched the power of one of the most popular Python single-cell analysis packages!

> *Note*: Colorbars aren't available in `seaborn`, only `pyplot` 😢 We wanted to emphasize `seaborn` today, but you can overcome this aesthetic difference simply using `plt.scatter()` and `plt.colorbar()` in place of `sns.scatterplot()`. Don't let that detract from your victory over Python!

# Moving on with Python
This marks the end of PyCamp (😢), but hopefully not the end of your Python journey (🐍).

The best way for you to get better with Python is to find a way to utilize it in your existing data analysis workflow. Although it might be a little bit of a slow start as you figure out how to build your analyses, practice makes perfect!

If you'd like tutorials that go beyond the `numpy`/`pandas`/`pyplot`/`seaborn` content covered in PyCamp, consider working through the materials available on [DataQuest](dataquest.io) or [Calm Code](https://calmcode.io/). Both websites provide walkthroughs and exercises for specific data tasks, and you may find them helpful for reinforcing and filling in any gaps that exist post-PyCamp.

Alternatively, if you're more of a book person, we recommend the following interactive books that are *also* published as physical books. You can order these via your favorite local/online bookshop.
* **Python for Data Analysis: Data Wrangling with Pandas, NumPy, and Python**: [online version](https://github.com/wesm/pydata-book); [publisher website](https://www.oreilly.com/library/view/python-for-data/9781491957653/)
* **Python Data Science Handbook**: [online version](https://github.com/jakevdp/PythonDataScienceHandbook), [publisher website](https://www.oreilly.com/library/view/python-data-science/9781491912126/)
* **Mastering Python for Bioinformatics: How to Write Flexible, Documented, Tested Python Code for Research Computing**: [online version](https://github.com/kyclark/biofx_python), [publisher website](https://www.oreilly.com/library/view/mastering-python-for/9781098100872/)

All of the packages below have extensive tutorials and documentation, so you'll be able to transition fairly seamlessly from the PyCamp style tutorial of "watch and repeat" to working on your own code!

* [SciPy](https://scipy.org/): A package that expands upon `numpy`, providing fundamental algorithms & functions for mathematical and statistical operations.
  * [scikit-image](https://scikit-image.org/): An affiliated package that provides image processing algorithms & functions.
  * [scikit-learn](https://scikit-learn.org/stable/): An affiliated package that provides machine learning algorithms and functions, one of which we used today for PCA!
* [Biopython](https://biopython.org/): A package for parsing and working with protein & sequence data, ranging from sequence parsing to alignment and querying of public databases.
* [statsmodels](https://www.statsmodels.org/stable/index.html): A module for statistical modeling and tests, expanding upon data structures from `numpy` and `pandas`.
* [keras](https://keras.io/): A relatively user-friendly package for deep learning.

# Challenge zone

The prompts below will require some more advance `pyplot`, `seaborn` and `pandas` techniques that we haven't covered explicitly in PyCamp. If you've finished early and would like to push yourself a bit, try them out: **note that we will not cover these solutions in session**.

## Multi-plot plots

Let's say that we want to examine many different genes and their expressions within the figure. We can create multi-plot figures using `pyplot` and `seaborn` together.

Multi-plot figures in `pyplot` are created with two types of objects.
* **Figure**: A Figure object is the "base" of your figure, akin to a blank canvas.
* **Subplot**: A Subplot is an object that only exists relative to a Figure: it specifies where your plotted data should go on the Figure.

Below is an example image that shows what a Figure object with one subplot looks like.

![fig_map](https://github.com/ccbskillssem/pythonbootcamp/raw/main/day_5/fig_map.png)

The simplest way to get started is to use the `plt.subplots()` function, which creates a grid of `(nrows, ncols)` subplots. This function takes several parameters:
* `nrows`: The number of rows in your grid of subplots.
* `ncols`: The number of columns in your grid of subplots.
* `figsize`: As described for `plt.figure()` earlier, a tuple of `(width, height)`.
* `layout`: Configures orientation of subplots. We recommend setting `layout = 'tight'` to minimize whitespace between subplots.

`plt.subplots()` will return a Figure and an array of Subplot objects, which are typically assigned to variables called `fig` and `axes`.

In [ ]:
# example: a Figure with four Subplots
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (4, 4), layout = 'tight')

print('Figure:', fig)
print('Subplots:', axes)

The code below creates a Figure with a 2 x 2 grid of Subplots. It takes advantage of two new functions/methods:
* `.flatten()`: A method for arrays, which converts `axes` from a 2 x 2 array to a 1 x 4 array that we can use in a `for` loop.
* `zip()`: A built-in function that "zips" together iterables (list, array, etc) of equivalent length. The first gene is matched with the first subplot, and so on and so forth.

In [ ]:
# example: what if we wanted to create a Figure of 2x2 subplots
# each containing a PCA plot colored by a different gene?
# this will take a moment to generate

plot_genes = ['Fxyd3', 'Gadd45g', 'Krt7', 'Col3a1']

fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (8, 8), layout = 'tight')
axes = axes.flatten()

for gene, ax in zip(plot_genes, axes.flatten()):
  sns.scatterplot(pca_with_expression.sort_values(gene), x = 'PC1', y = 'PC2',
                  hue = gene, ax = ax,
                  palette = 'viridis', linewidth = 0, s = 11, legend = None)
  ax.set_title(gene)

Using the above code as a skeleton, create a Figure with a subplot for each gene in `age_up` (or `age_down`). You will need to adjust the grid dimensions and Figure size.

We recommend making these changes as well:
* Use `.sort_values()`, as described earlier, to sort the DataFrame by the gene's expression values.
* Add a title to each subplot: at minimum, it should contain the gene name.
* Disable the legend in `sns.scatterplot()`: you can see how busy the plot is already!

In [ ]:
### write your code below ###
### plots will take a while to generate! ###


## Changes in relative cell proportions with age (Fig. 2E)

The Tabula Muris Senis dataset contains data derived from two different single-cell techniques:

| Technique   | Quantity of cells | Sensitivity |
| ----------- | ----------------- | ----------- |
| Microfluidic droplet-based sequencing      | +++       | +       |
| FACS-sorted plate-based sequencing   | +        | +++       |

Each technique has its own pros and cons: in this mini-project, we worked on the FACS data because its count matrix is smaller and less likely to overlab Colab's memory (and destroy your work in progress!)

Let's try to replicate the figure and analysis of Figure 2E ("Bladder cell"; "Bladder urothelial cell"), which uses the larger droplet sequencing dataset. The figure and relevant section from the paper are reprinted below for your convenience.

> *The cellular composition of each tissue type tends to vary with age. We investigated changes in the cellular composition of tissues for which data from at least three time points was available ...<br><br>The bladder shows pronounced changes in cell-type composition with age (Fig. 2e): whereas the mesenchymal compartment of this tissue decreases by a factor of three over the lifetime of the mouse (Fig. 2e, left), the urothelial compartment increases by a similar amount (Fig. 2e, right). The observation that the proportion of bladder urothelial cells increases with age is concordant with known age-related urothelial changes [25].*


<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41586-020-2496-1/MediaObjects/41586_2020_2496_Fig2_HTML.png?as=webp">

In [ ]:
# run this cell to download the droplet metadata: this should only take a few seconds
%%bash
wget https://cells.ucsc.edu/tabula-muris-senis/droplet/bladder/meta.tsv -O bladder-droplet.meta.tsv --no-check-certificate --no-verbose

In [ ]:
# use the appropriate pandas function to load in bladder-droplet.meta.tsv
# and assign it to a variable called metadata_droplet


Figure 2E shows that the relative proportion of bladder cells decreases with age, whereas the relative proportion of bladder urothelial cells increases with age. Both trends are statistically significant.

In order to create a plot-friendly DataFrame, you will need to do the following:
* Get group-based counts of cells by `age` and `cell_ontology_class` as a DataFrame. (Make sure it's a DataFrame and not a Series!)
* Create a new column with relative proportions of cell counts for each age group. **Make sure to check that the relative proportions for each sample sum up to 1.**
  * *Hint*: The `.transform()` method is very powerful for "filling in" values to each row that were originally calculated with grouped rows. Relevant StackOverflow answers for calculating relative proportions are likely to suggest that you use `.transform()`.
* Replace the strings in ages with `int` or `float` equivalents (ex. `3` instead of `3m`). This will make your data easier to plot.
* When you're confident that everything looks the way it should, reset the DataFrame index. This returns the data to a non-hierarchical format that `seaborn` and `pyplot` can parse.

In [ ]:
### write your code below ###


Finally, let's create the plot(s).

You'll need to use `sns.regplot()`, which will draw both a scatterplot and linear regression line. You can view the documentation for `sns.regplot()` [here](https://seaborn.pydata.org/generated/seaborn.regplot.html).

Notes:
* You can draw separate plots for `'Bladder cell'` and `'Bladder urothelial cell'` or use `plt.subplots()` to create a single Figure with two plots.
* You can alter the regression line style to a black dashed line using `line_kws = {'linestyle': 'dashed', 'color': 'black'}`.
* For this exercise, ignore the different scatter point colors.

In [ ]:
### write your code for the Bladder cell plot below ###


In [ ]:
### write your code for the Bladder urothelial cell plot below ###


As a final touch, we can repeat the statistical analysis described below.

> *The relative abundances of bladder cells ... and bladder urothelial cells ... change significantly with age. P < 0.05 and r2 > 0.7 for a hypothesis test with the null hypothesis that the slope is zero, using two-sided Wald test with t-distribution of the test statistic.*

We can perform this test using `scipy.stats.linregress()`. The documentation for this function is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html). You will need to perform this test once for `Bladder cells`, then again for `Bladder urothelial cells`.

In [ ]:
from scipy import stats

### write your code below ###
